In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML


In [2]:
%%R

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.2     ✔ tibble    3.3.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.2
✔ purrr     1.2.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


In [3]:
%%R

file.exists('~/Downloads/fema_data/fema_data.csv')
list.files('~/Downloads/fema_data/')


[1] "Fema_Data.csv"    "FEMA.ipynb"       "Untitled-2.ipynb"


In [4]:
%%R
df <- read.csv('~/Downloads/Fema_Data/Fema_Data.csv')
colnames(df)
head(df)

 declarationDate                                        disasterName stateCode
1      2024-01-07              SEVERE STORMS, FLOODING, AND TORNADOES        RI
2      2024-01-13 SEVERE STORMS, FLOODING, AND A POTENTIAL DAM BREACH        CT
3      2024-01-30  SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSLIDES        WV
4      2024-01-30                           SEVERE STORM AND FLOODING        NY
5      2024-01-30                           SEVERE STORM AND FLOODING        ME
6      2024-02-15         SEVERE WINTER STORM AND STRAIGHT-LINE WINDS        ND
      stateName   declarationRequestDate Days.Diff Quarter
1  Rhode Island 2023-11-09T00:00:00.000Z        58       1
2   Connecticut 2024-01-12T00:00:00.000Z         1       1
3 West Virginia 2023-10-25T00:00:00.000Z        67       1
4      New York 2023-11-29T00:00:00.000Z        62       1
5         Maine 2024-01-16T00:00:00.000Z        14       1
6  North Dakota 2024-01-26T00:00:00.000Z        20       1


In [6]:
%%R
install.packages("plotly", quiet = TRUE)
library(plotly)

# Calculate LOESS smooth with confidence intervals
smooth_data <- loess(Days.Diff ~ as.numeric(as.Date(declarationDate)), data = df)
smooth_fit <- predict(smooth_data, se = TRUE)
df$smooth <- smooth_fit$fit
df$se <- smooth_fit$se.fit

# Clean up declarationRequestDate to remove timestamp
df$declarationRequestDate_clean <- sub("T.*", "", df$declarationRequestDate)

# Create figure with data points
fig <- plot_ly() %>%
  
  # Confidence interval shading (upper)
  add_trace(
    x = as.Date(df$declarationDate),
    y = smooth_fit$fit + 1.96 * smooth_fit$se.fit,
    mode = "lines",
    line = list(color = "transparent"),
    fill = "tozeroy",
    fillcolor = "rgba(162, 59, 114, 0.1)",
    hoverinfo = "skip",
    showlegend = FALSE
  ) %>%
  
  # Confidence interval shading (lower)
  add_trace(
    x = as.Date(df$declarationDate),
    y = smooth_fit$fit - 1.96 * smooth_fit$se.fit,
    mode = "lines",
    line = list(color = "transparent"),
    fill = "tonexty",
    fillcolor = "rgba(162, 59, 114, 0.1)",
    hoverinfo = "skip",
    showlegend = FALSE
  ) %>%
  
  # Smooth line
  add_trace(
    data = df,
    x = ~as.Date(declarationDate),
    y = ~smooth,
    mode = "lines",
    type = "scatter",
    line = list(color = "#A23B72", width = 2),
    hoverinfo = "skip",
    showlegend = FALSE
  ) %>%
  
  # Data points with hover
  add_trace(
    data = df,
    x = ~as.Date(declarationDate), 
    y = ~Days.Diff,
    mode = "markers",
    type = "scatter",
    marker = list(size = 8, color = "#2D5016", opacity = 0.6),
    text = ~paste("Disaster: ", disasterName, "<br>",
                  "State: ", stateName, "<br>",
                  "Request Date: ", declarationRequestDate_clean, "<br>",
                  "Days to Approval: ", Days.Diff),
    hoverinfo = "text",
    hovertemplate = "%{text}<extra></extra>",
    showlegend = FALSE
  ) %>%
  
  # 2024 Event annotation markers (Blue)
  add_trace(
    x = c(as.Date("2024-03-01"), as.Date("2024-06-01")),
    y = c(max(df$Days.Diff, na.rm = TRUE) + 15, 
          max(df$Days.Diff, na.rm = TRUE) + 15),
    mode = "markers",
    type = "scatter",
    marker = list(size = 16, color = "#4169E1", opacity = 0.5, symbol = "diamond"),
    text = c("March 2024: DHS Appropriations Act adds $20.261 billion for the Disaster Relief Fund",
             "June 2024: Congress passed $500 million flood mitigation fund"),
    hoverinfo = "text",
    hovertemplate = "%{text}<extra></extra>",
    showlegend = FALSE
  ) %>%
  
  # 2025 Event annotation markers (Red)
  add_trace(
    x = c(as.Date("2025-05-01"), as.Date("2025-10-07")),
    y = c(max(df$Days.Diff, na.rm = TRUE) + 15, 
          max(df$Days.Diff, na.rm = TRUE) + 15),
    mode = "markers",
    type = "scatter",
    marker = list(size = 16, color = "#CC0000", opacity = 0.5, symbol = "diamond"),
    text = c("May 2025: Staff reduced by 1/3, $646M budget cut",
             "October 2025: FEMA canceled $11B in disaster payments"),
    hoverinfo = "text",
    hovertemplate = "%{text}<extra></extra>",
    showlegend = FALSE
  ) %>%
  
  layout(
    title = list(text = "<b>FEMA Disaster Declaration Processing Doubles in the Fourth Quarter of 2025</b><br><sub>Showing relationship between processing delays and major policy changes in 2024 and 2025</sub>",
    font=list(size=18)),
    xaxis = list(
      title = "Declaration Date of Disaster",
      zeroline = FALSE,
      showline = TRUE,
      showgrid = TRUE,
      gridcolor = "#e8e8e8"
    ),
    yaxis = list(
      title = "Processing Time in Days",
      zeroline = FALSE,
      showline = TRUE,
      showgrid = TRUE,
      gridcolor = "#e8e8e8"
    ),
    hovermode = "closest",
    plot_bgcolor = "white",
    paper_bgcolor = "white",
    font = list(family = "sans-serif", size = 12, color = "#333333"),
    margin = list(t = 100, r = 40, b = 60, l = 60),
    shapes = list(
      list(type = "line", x0 = as.Date("2024-03-01"), x1 = as.Date("2024-03-01"), y0 = 0, y1 = max(df$Days.Diff, na.rm = TRUE), line = list(dash = "dash", color = "#444444", width = 1)),
      list(type = "line", x0 = as.Date("2024-06-01"), x1 = as.Date("2024-06-01"), y0 = 0, y1 = max(df$Days.Diff, na.rm = TRUE), line = list(dash = "dash", color = "#444444", width = 1)),
      list(type = "line", x0 = as.Date("2025-05-01"), x1 = as.Date("2025-05-01"), y0 = 0, y1 = max(df$Days.Diff, na.rm = TRUE), line = list(dash = "dash", color = "#444444", width = 1)),
      list(type = "line", x0 = as.Date("2025-10-07"), x1 = as.Date("2025-10-07"), y0 = 0, y1 = max(df$Days.Diff, na.rm = TRUE), line = list(dash = "dash", color = "#444444", width = 1))
    )
  )

fig

No trace type specified:
  Based on info supplied, a 'scatter' trace seems appropriate.
  Read more about this trace type -> https://plotly.com/r/reference/#scatter
No trace type specified:
  Based on info supplied, a 'scatter' trace seems appropriate.
  Read more about this trace type -> https://plotly.com/r/reference/#scatter
